In [0]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import sklearn
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# data for 1%
data = pd.read_csv(base_dir + "1_training_final.csv")
data.info()

In [0]:
train, test = sklearn.model_selection.train_test_split(data, train_size=0.8, test_size=0.2)

Test shape  (752445, 67)
(3009778, 67)


In [0]:
train.to_csv(base_dir + "1_train_set.csv", index=False)
test.to_csv(base_dir + "1_test_set.csv", index=False)

## Random Forest Classifier

In [1]:
# Train and test set for 1% of the data
chunksize1 = 10000
train = pd.read_csv(base_dir + "/1percent/" + "1_train_set_RF.csv", chunksize= chunksize1)
test = pd.read_csv(base_dir + "/1percent/" + "1_test_set_RF.csv")

y_test = test.hotel_cluster
X_test = test.drop(columns=["hotel_cluster"])

In [0]:
# GridSearch to find optimal parameters

rf = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV
# Create the parameter grid 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 150, 200],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 11, 12],
    'n_estimators': [200, 300, 1000, 1500, 2000]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, verbose = 2)

# Fit the grid search to the data (by chunks)
count1 = 0
for chunk in train:
    y = chunk.hotel_cluster
    x = chunk.drop(columns=["hotel_cluster"])
    grid_search.fit(x, y)
    count1 = count1 + chunksize1
    print('%d rows trained' % count)



In [0]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 200,
 'max_features': 4,
 'min_samples_leaf': 4,
 'min_samples_split': 11,
 'n_estimators': 1500}

In [2]:
# Define model and train RF
clf = RandomForestClassifier(n_estimators=1500, max_depth=200, max_features=4, min_samples_leaf=4, min_samples_split=11, bootstrap=True)
count = 0

for chunk in train:
    y = chunk.hotel_cluster
    x = chunk.drop(columns=["hotel_cluster"])
    clf.fit(x, y)
    count = count + chunksize1
    print('%d rows trained' % count)

In [3]:
# Predict class probabilities
pred = clf.predict_proba(X_test)

In [ ]:
# function to measure performance MAP@5
def map5eva(preds, actual):
    predicted = preds.argsort(axis=1)[:, -np.arange(5)]
    metric = 0
    for i in range(5):
        metric += np.sum(actual==predicted[:, i])/(i+1)
    metric /= actual.shape[0]
    return "MAP@5", -metric

In [4]:
map5eva(pred, y_test)